# Beta Bank Customer Churn Prediction

### Descarga y prepara los datos.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix

# Cargar de datos y visualizacion
data = pd.read_csv('/datasets/Churn.csv')
print("Vista previa de los datos:")
print(data.head())
print("\nInformacion sobre los datos:")
print(data.info())

#Quitamos las columnas irrelevantes
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

#Codificacion con dummies de variables categoricas
data = pd.get_dummies(data, drop_first=True)

#Correccion de valores nulos e infinitos
data.fillna(0, inplace=True)
data.replace([np.inf, -np.inf], 0, inplace=True)

#Separamos Features y target
features = data.drop('Exited', axis=1)
target = data['Exited']

#Dividimos el conjunto de datos
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.2, stratify=target, random_state=42
)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, stratify=target_train, random_state=42
)

#Escalados de features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_valid_scaled = scaler.transform(features_valid)
features_test_scaled = scaler.transform(features_test)

Vista previa de los datos:
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826

### Examina el equilibrio de clases. Entrena el modelo sin tener en cuenta el desequilibrio. 

Se procedio a realizar el seleccionamiento de caracteristicas con importacia en Random Forest.

El modelo nos ha señalado 11 variables clave que son de importancia y podrian influir en la prediccion de abandono de clientes, entre las varibles se encuentran Age, Balance, CreditScore y Tenure el cual juegan un papel fundamental para lo que es la retencion de clientes. Adicionalmente, el adicionar las variables Geography_Germany y Geography_Spain nos dice que la ubicacion geografica si es una de las variables que realmente influye en la tasa de abandono.

In [32]:
#Modelo para analizar importancia de caracteristicas
rf_temp = RandomForestClassifier(random_state=42, class_weight='balanced')
rf_temp.fit(features_train_scaled, target_train)

#Obtencion de importancia de caractersticas
importances = pd.Series(rf_temp.feature_importances_, index=features.columns)
importances = importances.sort_values(ascending=False)

#Filtramos caracteristicas con importancia mayor a 0.01
selected_features = importances[importances > 0.01].index.tolist()
features_train_selected = features_train[selected_features]
features_valid_selected = features_valid[selected_features]
features_test_selected = features_test[selected_features]

print(f"Caracteristicas seleccionadas: {selected_features}")

Caracteristicas seleccionadas: ['Age', 'Balance', 'EstimatedSalary', 'CreditScore', 'NumOfProducts', 'Tenure', 'IsActiveMember', 'Geography_Germany', 'Gender_Male', 'HasCrCard', 'Geography_Spain']


### Mejora la calidad del modelo.

Se realizo la optimizacion de hiperparametros en Random Forest

In [33]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_search = GridSearchCV(rf, param_grid, scoring='f1', cv=3, n_jobs=-1)
grid_search.fit(features_train_selected, target_train)

best_rf = grid_search.best_estimator_
print("Mejores hiperparametros encontrados:", grid_search.best_params_)

Mejores hiperparametros encontrados: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}


### Prueba final.

Tambien se ajusto el umbral de decision para poder estar mar cerca de de F1 requerido en el proyecto,ya que el umbral optimo encontrado 0.55 balancea mejor la precision y el recall, reduciendo los falsos negativos (FN) sin comprometer en exceso la precision.

In [34]:
probs_valid = best_rf.predict_proba(features_valid_selected)[:, 1]

optimal_thresholds = np.arange(0.55, 0.6, 0.01)
best_f1 = 0
best_threshold = 0.5

for threshold in optimal_thresholds:
    preds_adjusted = (probs_valid > threshold).astype(int)
    f1 = f1_score(target_valid, preds_adjusted)
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = threshold

print(f"Mejor umbral ajustado: {best_threshold}")
print(f"F1 Score en validacion: {best_f1}")

Mejor umbral ajustado: 0.55
F1 Score en validacion: 0.6243523316062175


Ya finalmente se realizo la evaluacion final del conjunton de pureba.

Adicionalmente, en la evaluacion final el F1 Score en validacion (0.624) nos dice que hay un buen equilibrio entre precision y recall. 

F1 Score en prueba fina (0.617) se llega a mantener estable, por el otro lado en el AUC-ROC (0.86) nos llega demostrar una buena capacidad del modelo para distinguir entre clientes que abandonan y los que permanecen.

In [35]:
probs_test = best_rf.predict_proba(features_test_selected)[:, 1]
preds_test_adjusted = (probs_test > best_threshold).astype(int)

print("Resultado final en conjunto de prueba")
print("F1 Score:", f1_score(target_test, preds_test_adjusted))
print("AUC-ROC:", roc_auc_score(target_test, probs_test))

print("Matriz de confusion:")
print(confusion_matrix(target_test, preds_test_adjusted))

Resultado final en conjunto de prueba
F1 Score: 0.6171875
AUC-ROC: 0.8637528129053553
Matriz de confusion:
[[1469  124]
 [ 170  237]]


# Conclusion

Con este modelo podemos ofrecer una prediccion confiable para nuestro cliente Beta Bank para que pueda estar anuente sobre la retencion de sus clientes, con un F1 Score por arriba de la meta solicitada (0.59) y un AUC-ROC elevado (0.86). Ahora bien, la combinacion de seleccion de caracteristicas optimizada, ajuste de hiperparametros y correccion del umbral nos ha permitido tener una mejora sustancial en la precision del modelo, asegurando una clasificacion mas efectiva en si.

Si bien tenemos falsos negativos en la prediccion con el ajuste del umbral ha reducido el impacto y lo cual mantiene una buena capacidad para identificar clientes en riesgo de abandono, por lo tanto que nuestro cliente(Beta Bank) pueda diseñar un plan de accion estrategico enfocado en la retencion en los perfiles que potencialmente podrian ser propensos a abandonar, dandole un buen margen de tiempo de reaccion ante algun abandono.
